In [ ]:
!pip install pydicom

In [ ]:
!pip install opencv-python-headless[app]

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import os
import glob
import cv2
from PIL import Image
from torch.utils.data import Dataset, DataLoader
from albumentations import HorizontalFlip, VerticalFlip, Rotate
from tqdm import tqdm
import torch.nn.functional as F
import matplotlib.image as mpimg
import albumentations as A
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, random_split
from sklearn.metrics import jaccard_score
from tqdm import tqdm
import nibabel as nib
import albumentations as A
import torch
import numpy as np
from torch.utils.data import Dataset
import nibabel as nib
import albumentations as A
import pydicom
import cv2
from PIL import Image
from torchvision.transforms import ToTensor, Normalize

height,width = (256, 256)


torch.cuda.is_available()

In [2]:
import random

# Dönüştürülecek dosyaların bulunduğu klasör yolu
folder_path = "/home/sinem/Masaüstü/kod/ornek/sinem/images/hasta_no/"

# Klasördeki tüm .dcm dosyalarını seç
files = [f for f in os.listdir(folder_path) if f.endswith('.dcm')]

# Her bir dosyayı dönüştür
for file in files:
    # Dosyayı oku
    ds = pydicom.dcmread(os.path.join(folder_path, file))
    
    # Piksel verilerini al ve boyutları döndür
    pixel_array = ds.pixel_array
    shape = pixel_array.shape

    # Piksel verilerini uint8 veri tipine dönüştür
    pixel_array = (pixel_array / np.max(pixel_array) * 255).astype('uint8')

    # Piksel verilerini Image nesnesine dönüştür
    im = Image.fromarray(pixel_array)
    
    # Dosya adını ve uzantısını değiştir
    filename = os.path.splitext(file)[0]
    new_filename = filename + '.jpg'
    
    # Dönüştürülmüş dosyayı kaydet
    im.save(os.path.join(folder_path, new_filename))

# Dönüştürülen dosyalardan birini göster
random_file = random.choice([f for f in os.listdir(folder_path) if f.endswith('.jpg')])
img = Image.open(os.path.join(folder_path, random_file))
img.show()

In [30]:
# Fotoğrafı oku
image = cv2.imread('/home/sinem/Masaüstü/kod/ornek/sinem/images/hasta_no/LMLO.jpg')

# Fotoğrafın boyutlarını al
height, width, channels = image.shape

# Dikdörtgenin köşe koordinatlarını belirle
top_left_x = np.random.randint(0, width // 2)
top_left_y = np.random.randint(0, height // 2)
bottom_right_x = np.random.randint(width // 2, width)
bottom_right_y = np.random.randint(height // 2, height)

# Mavi dikdörtgeni fotoğrafın üzerine çiz
color = (255, 0, 0)  # BGR formatında mavi renk
thickness = 2
cv2.rectangle(image, (top_left_x, top_left_y), (bottom_right_x, bottom_right_y), color, thickness)

# Dikdörtgenin çizildiği fotoğrafı göster ve kaydet
cv2.imshow('Image with Blue Rectangle', image)
cv2.imwrite('/home/sinem/Masaüstü/kod/ornek/sinem/images/hasta_no/new_image-2.jpg', image)
cv2.waitKey(5000) # 5 saniye bekleyin (5000 milisaniye)
cv2.destroyAllWindows()

In [ ]:
imgs = pydicom.dcmread('/home/sinem/Masaüstü/kod/ornek/sinem/images/hasta_no/LMLO.dcm').pixel_array
masks = nib.load('/home/sinem/Masaüstü/kod/ornek/mission.nii.gz')
masks=masks.get_fdata()
masks=masks[:,:,3]
#masks[masks==2]=0
#masks[masks==3]=0
masks=np.flip(masks)
masks=np.rot90(masks)
masks=np.fliplr(masks)
img=np.invert(imgs)


plt.imshow(imgs,cmap="gray")
plt.imshow(masks,cmap="inferno",alpha=0.5)
plt.show()

In [ ]:
# Dicom dosyasının yolu
filename = '/home/sinem/Masaüstü/kod/ornek/sinem/images/hasta_no/LMLO.dcm'

# Dicom dosyasını yükleme
imgs = pydicom.dcmread(filename).pixel_array

# Maske dosyasının yolu
mask_filename = '/home/sinem/Masaüstü/kod/ornek/mission.nii.gz'

# Maske dosyasını yükle
mask_nii = nib.load(mask_filename)

# Maske verilerini NumPy dizisine dönüştür
mask = np.array(mask_nii.get_fdata()[:, :, 3])

# Maskeyi çevir ve döndür
mask = np.flip(mask, axis=1)
mask = np.rot90(mask)


# Dicom görüntüsünü ters çevir
img = np.invert(imgs)

# Maske ve görüntüyü birleştirip görüntüle
plt.imshow(img, cmap='gray')
plt.imshow(mask, cmap='inferno', alpha=0.5)

# Maske verilerindeki pozitif koordinatların indekslerini bul
positive_indices = np.where(mask == 1)

# Pozitif koordinatların minimum ve maksimum değerlerini hesapla
min_row, min_col = np.min(positive_indices, axis=1)
max_row, max_col = np.max(positive_indices, axis=1)

# Koordinatları çevreleyen dikdörtgenin merkezini hesapla
mid_row, mid_col = (min_row + max_row) / 2, (min_col + max_col) / 2

# Görüntüleme
plt.imshow(img, cmap='gray')
plt.imshow(mask, cmap='inferno', alpha=0.5)
plt.scatter(mid_col, mid_row, color='red')
plt.show()

In [4]:

# Resmi Gri Tonlama ile Yükleme
image = cv2.imread('/home/sinem/Masaüstü/kod/ornek/sinem/images/hasta_no/new_image-2.jpg', cv2.IMREAD_GRAYSCALE)

# Renk Eşikleme Uygulama
_, threshold = cv2.threshold(image, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)

# Morfolojik İşlemler
kernel = np.ones((4,4), np.uint8)
dilation = cv2.dilate(threshold, kernel, iterations=1)

# Vücudun Konturunu Bul
contours, hierarchy = cv2.findContours(dilation, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
if not contours:
    print("Vücut konturu bulunamadı.")
    exit()

# En Büyük Konturu Seçin
contour = max(contours, key=cv2.contourArea)

# Konturun Merkez Noktasını Hesaplayın
M = cv2.moments(contour)

# Sıfıra bölme hatasını önlemek için kontrol yapın
if M['m00'] == 0:
    print("Sıfıra bölme hatası! Merkez noktası hesaplanamadı.")
    exit()

center_x = int(M['m10'] / M['m00'])
center_y = int(M['m01'] / M['m00'])

# Beyaz Piksellerin Koordinatlarını Bul
white_pixels = np.column_stack(np.where(threshold > 0))

# Beyaz Piksellerin Ortalama Koordinatını Hesapla
center_x, center_y = np.mean(white_pixels, axis=0)[::-1]
center = (int(center_x), int(center_y))

# Merkez Noktasını Kırmızı Bir Daire ile İşaretleyin
image_with_circle = cv2.circle(cv2.cvtColor(image, cv2.COLOR_GRAY2BGR), center, radius=10, color=(0, 0, 255), thickness=-1)

# Sonuçları göster
cv2.imshow('image', image)
cv2.imshow('image_with_circle', image_with_circle)
cv2.imwrite('mission.jpg', image_with_circle)
cv2.waitKey(5000) # 5 saniye bekleyin (5000 milisaniye)
cv2.destroyAllWindows()

In [ ]:
# Görüntüyü yükle
img = cv2.imread('/home/sinem/Masaüstü/kod/ornek/sinem/images/hasta_no/new_image.jpg')

# Görüntüyü RGB renk uzayına dönüştür
img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

# Mavi renkli pikselleri tanımlayan bir maske oluştur
lower_blue = np.array([100, 0, 0]) # Mavi renk aralığı alt sınırı
upper_blue = np.array([255, 50, 50]) # Mavi renk aralığı üst sınırı
mask = cv2.inRange(img_rgb, lower_blue, upper_blue)

# Gürültüyü azaltmak için Gauss filtresi uygula
mask = cv2.GaussianBlur(mask, (5, 5), 0)

# Dış kenarları (kontürleri) bul
contours, hierarchy = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# En büyük dış kenarı seç
max_contour = max(contours, key=cv2.contourArea, default=[])

# Eğer en büyük kontur boş değilse
if max_contour:
    # Konturun dış dikdörtgenini hesapla
    x, y, w, h = cv2.boundingRect(max_contour)
    # Dikdörtgeni çiz
    cv2.rectangle(img, (x, y), (x+w, y+h), (0, 0, 255), 2)
else:
    print("Dikdörtgen bulunamadı.")

# Dikdörtgenin merkez noktasını hesapla
cx, cy = x + int(w / 2), y + int(h / 2)

# Merkez noktayı kırmızı bir daire ile işaretle
cv2.circle(img, (cx, cy), 5, (0, 0, 255), -1)

# Dikdörtgeni yeşil bir çerçeve ile çevrele
cv2.rectangle(img, (x, y), (x + w, y + h), (0, 255, 0), 2)

# Sonuçları göster
cv2.imshow('image', image)
cv2.imshow('image_with_circle', image_with_circle)
cv2.imwrite('mission.jpg', image_with_circle)
cv2.waitKey(5000) # 5 saniye bekleyin (5000 milisaniye)
cv2.destroyAllWindows()

In [2]:
# Resmi Yükle
image = cv2.imread('/home/sinem/Masaüstü/kod/ornek/sinem/images/hasta_no/new_image.jpg')

# Griye Dönüştür
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# Kenar Algılama
edges = cv2.Canny(gray, 50, 150, apertureSize=3)

# Hough Dönüşümü İle Dikdörtgeni Bul
lines = cv2.HoughLinesP(edges, rho=1, theta=np.pi/180, threshold=100, minLineLength=100, maxLineGap=10)
if lines is None:
    print("Dikdörtgen bulunamadı.")
    exit()

# Dikdörtgenin Koordinatlarını Hesaplayın
x_min, y_min = np.inf, np.inf
x_max, y_max = 0, 0
for line in lines:
    x1, y1, x2, y2 = line[0]
    x_min = min(x_min, x1, x2)
    y_min = min(y_min, y1, y2)
    x_max = max(x_max, x1, x2)
    y_max = max(y_max, y1, y2)

# Dikdörtgeni Çizin
cv2.rectangle(image, (x_min, y_min), (x_max, y_max), (0, 0, 255), 2)

# Dikdörtgenin Orta Noktasını Hesaplayın
center_x = int((x_min + x_max) / 2)
center_y = int((y_min + y_max) / 2)

# Orta Noktayı Kırmızı Bir Daire ile İşaretleyin
center = (center_x, center_y)
cv2.circle(image, center, radius=20, color=(0, 0, 255), thickness=-1)

# Sonuçları Gösterin
cv2.imshow('image', image)
cv2.imwrite('mission.jpg', image)
cv2.waitKey(5000) # 5 saniye bekleyin (5000 milisaniye)
cv2.destroyAllWindows()

In [ ]:
# Resmi Yükle
image = cv2.imread('/home/sinem/Masaüstü/kod/ornek/sinem/images/hasta_no/new_image-2.jpg')

# Gaussian Bulanıklaştırma Uygula
blurred = cv2.GaussianBlur(image, (5, 5), 0)

# Canny Kenar Algılama Uygula
edges = cv2.Canny(blurred, 50, 150)

# Morfolojik Kapama İşlemi Uygula
kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (3,3))
closed = cv2.morphologyEx(edges, cv2.MORPH_CLOSE, kernel)

# Konturları Bul
contours, hierarchy = cv2.findContours(closed, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# En Büyük Konturu Bul ve Dikdörtgenle Sınırla
largest_contour = max(contours, key=cv2.contourArea)
rect = cv2.minAreaRect(largest_contour)
box = cv2.boxPoints(rect)
box = np.int0(box)

# Dikdörtgeni Çiz
cv2.drawContours(image, [box], 0, (0, 0, 255), 2)

# Dikdörtgenin Orta Noktasını Hesapla
center_x = int((box[0][0] + box[2][0]) / 2)
center_y = int((box[0][1] + box[2][1]) / 2)

# Orta Noktayı Kırmızı Bir Daire ile İşaretleyin
center = (center_x, center_y)
cv2.circle(image, center, radius=20, color=(0, 0, 255), thickness=-1)

# Sonucu Göster
cv2.imshow('image', image)
cv2.imwrite('mission.jpg', image)
cv2.waitKey(5000)
cv2.destroyAllWindows()


In [36]:
# Resmi Yükle
image = cv2.imread('/home/sinem/Masaüstü/kod/ornek/sinem/images/hasta_no/new_image-2.jpg')

# Griye Dönüştür
#gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# Kenar Algılama
edges = cv2.Canny(gray, 60, 120, apertureSize=3)

# Hough Dönüşümü İle Çizgileri Bul
lines = cv2.HoughLinesP(edges, 1, np.pi/180, 50, minLineLength=100, maxLineGap=10)

# Çizgi noktalarını listeye dönüştür
points = []
for line in lines:
    x1, y1, x2, y2 = line[0]
    points.append((x1, y1))
    points.append((x2, y2))

# Dikdörtgenin Koordinatlarını Hesaplayın
x1, y1, x2, y2 = lines[0][0]
for line in lines:
    x1_, y1_, x2_, y2_ = line[0]
    x1 = max(x1, x1_)
    y1 = max(y1, y1_)
    x2 = min(x2, x2_)
    y2 = min(y2, y2_)

# Dikdörtgen sınırlayıcı kutusunu hesaplayın
x, y, w, h = cv2.boundingRect(np.array(points))

# Yükseklik ve genişlik oranlarını kontrol edin ve düzeltin
aspect_ratio = h/w
if aspect_ratio > 1.5:
    new_h = int(w * 1.5)
    y = y + int((h - new_h)/2)
    h = new_h
elif aspect_ratio < 0.5:
    new_w = int(h / 0.5)
    x = x + int((w - new_w)/2)
    w = new_w

# Dikdörtgeni Çizin
cv2.rectangle(image, (x, y), (x + w, y + h - 30), (255, 0, 0), 2)

# Orta Noktayı Kırmızı Bir Daire ile İşaretleyin
center = (center_x, center_y)
cv2.circle(image, center, radius=20, color=(0, 0, 255), thickness=-1)

# Sonuçları Gösterin
cv2.imshow('image', image)
cv2.imwrite('mission.jpg', image)
cv2.waitKey(5000) # 5 saniye bekleyin (5000 milisaniye)
cv2.destroyAllWindows()

In [9]:
# Resmi Yükle
image = cv2.imread('/home/sinem/Masaüstü/kod/ornek/sinem/images/hasta_no/new_image-2.jpg')

# Griye Dönüştür
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# Kenar Algılama
edges = cv2.Canny(gray, 60, 120, apertureSize=3)

# Hough Dönüşümü İle Çizgileri Bul
lines = cv2.HoughLinesP(edges, 1, np.pi/180, 50, minLineLength=100, maxLineGap=10)

# Mavi dikdörtgeni bulmak için sınırlayıcı kutu boyutlarını saklayacak değişkenler
max_x, max_y, max_w, max_h = 0, 0, 0, 0

# Diğer dikdörtgenleri filtrele
for line in lines:
    x1, y1, x2, y2 = line[0]
    # Line'ın rengini tespit etmek için iki uç noktasındaki renklerin ortalamasını hesaplayın
    color = np.mean(image[y1:y2, x1:x2], axis=(0, 1))
    # Mavi renkteki dikdörtgeni bulmak için mavi rengin yoğunluğunu kontrol edin
    if color[0] > color[1] and color[0] > color[2]:
        if x1 > x2:
            x1, x2 = x2, x1
        if y1 > y2:
            y1, y2 = y2, y1
        w, h = x2 - x1, y2 - y1
        aspect_ratio = h / w
        # Yükseklik ve genişlik oranlarını kontrol edin ve düzeltin
        if aspect_ratio > 1.5:
            new_h = int(w * 1.5)
            y1 = y1 + int((h - new_h)/2)
            h = new_h
        elif aspect_ratio < 0.5:
            new_w = int(h / 0.5)
            x1 = x1 + int((w - new_w)/2)
            w = new_w
        # Mavi dikdörtgenin sınırlayıcı kutusu boyutlarını güncelle
        if w * h > max_w * max_h:
            max_x, max_y, max_w, max_h = x1, y1, w, h

# Mavi dikdörtgenin sınırlayıcı kutusunu çizin
cv2.rectangle(image, (max_x, max_y), (max_x + max_w, max_y + max_h - 30), (255, 0, 0), 2)

# Orta Noktayı Kırmızı Bir Daire ile İşaretleyin
center_x, center_y = max_x + max_w // 2, max_y + max_h // 2 - 15 # Yeni ekleme: -15
center = (center_x, center_y)
cv2.circle(image, center, radius=20, color=(0, 0, 255), thickness=-1)

# Sonuçları Gösterin
cv2.imshow('image', image)
cv2.imwrite('mission.jpg', image)
cv2.waitKey(5000) # 5
cv2.destroyAllWindows()

In [ ]:
# Resmi Yükle
image = cv2.imread('/home/sinem/Masaüstü/kod/ornek/sinem/images/hasta_no/new_image-2.jpg')

# Griye Dönüştür
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# Kenar Algılama
edges = cv2.Canny(gray, 60, 120, apertureSize=3)

# Hough Dönüşümü İle Çizgileri Bul
lines = cv2.HoughLinesP(edges, 1, np.pi/180, 50, minLineLength=100, maxLineGap=10)

# Mavi dikdörtgeni bulmak için sınırlayıcı kutu boyutlarını saklayacak değişkenler
max_x, max_y, max_w, max_h = 0, 0, 0, 0

# Diğer dikdörtgenleri filtrele
for line in lines:
    x1, y1, x2, y2 = line[0]
    # Line'ın rengini tespit etmek için iki uç noktasındaki renklerin ortalamasını hesaplayın
    color = np.mean(image[y1:y2, x1:x2], axis=(0, 1))
    # Mavi renkteki dikdörtgeni bulmak için mavi rengin yoğunluğunu kontrol edin
    if color[0] > color[1] and color[0] > color[2]:
        if x1 > x2:
            x1, x2 = x2, x1
        if y1 > y2:
            y1, y2 = y2, y1
        w, h = x2 - x1, y2 - y1
        aspect_ratio = h / w
        # Yükseklik ve genişlik oranlarını kontrol edin ve düzeltin
        if aspect_ratio > 1.5:
            new_h = int(w * 1.5)
            y1 = y1 + int((h - new_h)/2)
            h = new_h
        elif aspect_ratio < 0.5:
            new_w = int(h / 0.5)
            x1 = x1 + int((w - new_w)/2)
            w = new_w
        # Mavi dikdörtgenin sınırlayıcı kutusu boyutlarını güncelle
        if w * h > max_w * max_h:
            max_x, max_y, max_w, max_h = x1, y1, w, h

# Mavi dikdörtgenin sınırlayıcı kutusunu çizin
cv2.rectangle(image, (max_x, max_y), (max_x + max_w, max_y + max_h - 30), (255, 0, 0), 2)

# Orta Noktayı Kırmızı Bir Daire ile İşaretleyin
center = (center_x, center_y)
cv2.circle(image, center, radius=20, color=(0, 0, 255), thickness=-1)

# Sonuçları Gösterin
cv2.imshow('image', image)
cv2.imwrite('mission_1.jpg', image)
cv2.waitKey(5000) # 5 saniye bekleyin (5000 milisaniye)
cv2.destroyAllWindows()


In [ ]:

# Resmi Yükle
image = cv2.imread('/home/sinem/Masaüstü/kod/ornek/sinem/images/hasta_no/new_image-2.jpg')

# Griye Dönüştür
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# Kenar Algılama
edges = cv2.Canny(gray, 60, 120, apertureSize=3)

# Hough Dönüşümü İle Çizgileri Bul
lines = cv2.HoughLinesP(edges, 1, np.pi/180, 50, minLineLength=100, maxLineGap=10)

# Mavi dikdörtgeni bulmak için sınırlayıcı kutu boyutlarını saklayacak değişkenler
max_x, max_y, max_w, max_h = 0, 0, 0, 0

# Diğer dikdörtgenleri filtrele
for line in lines:
    x1, y1, x2, y2 = line[0]
    # Line'ın rengini tespit etmek için iki uç noktasındaki renklerin ortalamasını hesaplayın
    color = np.mean(image[y1:y2, x1:x2], axis=(0, 1))
    # Mavi renkteki dikdörtgeni bulmak için mavi rengin yoğunluğunu kontrol edin
    if color[0] > color[1] and color[0] > color[2]:
        if x1 > x2:
            x1, x2 = x2, x1
        if y1 > y2:
            y1, y2 = y2, y1
        w, h = x2 - x1, y2 - y1
        aspect_ratio = h / w
        # Yükseklik ve genişlik oranlarını kontrol edin ve düzeltin
        if aspect_ratio > 1.5:
            new_h = int(w * 1.5)
            y1 = y1 + int((h - new_h)/2)
            h = new_h
        elif aspect_ratio < 0.5:
            new_w = int(h / 0.5)
            x1 = x1 + int((w - new_w)/2)
            w = new_w
        # Mavi dikdörtgenin sınırlayıcı kutusu boyutlarını güncelle
        if w * h > max_w * max_h:
            max_x, max_y, max_w, max_h = x1, y1, w, h

# Mavi dikdörtgenin sınırlayıcı kutusunu çizin
cv2.rectangle(image, (max_x, max_y), (max_x + max_w, max_y + max_h - 30), (255, 0, 0), 2)

# Dikdörtgenin orta noktasını hesaplayın
center_x, center_y = max_x + max_w // 2, max_y + max_h // 2 - 15 # Yeni ekleme: -15

# Dikdörtgenin içindeki siyah pikselleri bulun
mask = np.zeros(gray.shape, dtype=np.uint8)
mask[max_y:max_y+max_h, max_x:max_x+max_w] = gray[max_y:max_y+max_h]

#Dikdörtgenin içindeki siyah pikselleri bulun (devam)
masked_gray = cv2.bitwise_and(gray, gray, mask=mask)

#Treshold ile ikili görüntü elde edin
ret, binary = cv2.threshold(masked_gray, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

#Görüntüyü dilate edin
kernel = np.ones((3,3),np.uint8)
dilated = cv2.dilate(binary, kernel, iterations=2)

#Dilate edilmiş görüntüden konturları bulun
contours, hierarchy = cv2.findContours(dilated, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

#En büyük konturu bulun
max_contour = max(contours, key=cv2.contourArea)

#Konturun dışına sığdırılmış minimum dikdörtgeni hesaplayın
rect = cv2.minAreaRect(max_contour)

#Dikdörtgenin kenarlarını bulun
box = cv2.boxPoints(rect)
box = np.int0(box)

#Dikdörtgeni çizin
cv2.drawContours(image,[box],0,(0,0,255),2)

#Dikdörtgenin orta noktasını hesaplayın
M = cv2.moments(max_contour)
center_x2 = int(M["m10"] / M["m00"])
center_y2 = int(M["m01"] / M["m00"])

#Dikdörtgenin orta noktasını çizin
cv2.circle(image, (center_x2, center_y2), 5, (0, 255, 0), -1)

#İşlenmiş görüntüyü gösterin
cv2.imshow("Processed Image", image)

# Sonuçları Gösterin
cv2.imshow('image', image)
cv2.imwrite('mission_2.jpg', image)
cv2.waitKey(5000) # 5 saniye bekleyin (5000 milisaniye)
cv2.destroyAllWindows()